# Data-Distributor

The `DataDistributor` essentially represents a `DataPubSubSystem`. It distributes data over the NoPE network (or internally). Data is manipulated via JSON pointers (these are structured like MQTT topics; see above). In order to inform systems about changes, `subscribers` can be informed about changes like in MQTT. The wildcards of MQTT can be used to get the relevant information. I.e.:
- The `DataDistributor` contains a `root` data object.
    - The `root` data object can be modified via paths (similar to topics) (see methods `patternBasedPush` or `pushData`)
- If the data object is manipulated, all changes are automatically forwarded to the interested `subscribers`.
- The naming of the methods corresponds to the push and pull principle 

## DataPubSubSystem

A data based publish and subscribe system.
It extends the PubSubSystem by providing the methods and properties:
- `pushData` to push data into the system.
- `pullData` to pull data from the system. It always returns the current data or the default value if there is no data under the specified path.
- `patternbasedPullData` to fetch data with a specific pattern. See the example for details.
- `patternBasedPush` to push data with a given pattern into the system.
- `data`: direct access to the `root` data object. This object is continuously present and always contains the most recent data manipulated by changes (see `patternBasedPush` or `pushData`).

## Interact with the DataDistributor:

Firstly we have to create a Dispatcher, that we are using to access the `eventDistributor`

In [1]:
// First lets install nope using npm
const nope = require("../dist-nodejs/index.nodejs")

// Create a communicator:
// We will use the event layer (which just runs internally)
const communicator = nope.getLayer("event");

// Lets create our dispatcher

// 1. Dispatcher simulates our local system
const localDispatcher = nope.dispatcher.getDispatcher({
  communicator,
  id: "local"
}, {
  singleton: false,
  useBaseServices: false
});

> For Jupyter we need an extra async wrapper to wait for initalizing the dispatcher:

see here for the details in Jupyter: https://n-riesco.github.io/ijavascript/doc/async.ipynb.html

In [2]:
$$.async();
// Lets wait for our element to be ready.
localDispatcher.ready.waitFor().then($$.done);

true

In [3]:
// Create a short cut:
const dataDistributor = localDispatcher.dataDistributor;

We are now ready to manipulate some data in the system:

In [4]:
// Now we manipulate the data:
dataDistributor.pushData("", { robot: {status: {state:"waiting", speed:0}}});
dataDistributor.data

{ robot: { status: { state: 'waiting', speed: 0 } } }

## Subscribing to data Changes

We are now interessed in data changes. We want allways be informed about the status of a specific object.

Therefore we create an subscription of on the topic `robot/status`. By subscribing, we will immediately receive an update containing the current status:

In [5]:
const observer = dataDistributor.registerSubscription("robot/status", (data) => {
    console.log("The Robot-Status is",data)
})

The Robot-Status is { state: 'waiting', speed: 0 }


We now simulate a change of the robots speed:

In [6]:
// We create an extra Element describing the status.
// We use an observable for that purpose

const robotStatus = new nope.NopeObservable();
robotStatus.setContent({state:"waiting", speed:0});

dataDistributor.register(robotStatus, {
    mode: "publish",
    topic: "robot/status"
});

// Now we adapt the status

robotStatus.getContent().speed = 1;
robotStatus.getContent().state = "moving";

The Robot-Status is { state: 'waiting', speed: 0 }


'moving'

As we can see, we didnt receive an update. Thats based on the behavior of the observable. We have to trigger an update manually right now, or must use the method `setContent` instead of adapting the data object directly.

In [11]:
robotStatus.forcePublish();

// Alternative:
robotStatus.setContent({state:"moving", speed:1});

The Robot-Status is { state: 'moving', speed: 1 }


false

But why didn't receive 2 updates? we used the method `forcePublish` and `setContent` and assigned a new status-object, but didnt receive 2 notifications. This is related to the behavior of observables. They only emit changes, if the data really changed (what wasnt the case)

## Accessing specific datapoints

If we want to manipulate data we can use the method `pushData`. Lets add an additional status of an sensor:

In [12]:
dataDistributor.pushData("sensor", {status: {state:"waiting"}});

The **root** object of the dataDistributor can be read with the property `data`

In [13]:
dataDistributor.data

{
  robot: { status: { state: 'moving', speed: 1 } },
  sensor: { status: { state: 'waiting' } }
}

If we want to access specific data points inside of the dataDistributor, we can use the function `pullData` or `patternBasedPullData`

In [14]:
// pull the data of the sensor:
dataDistributor.pullData("sensor",false)

{ status: { state: 'waiting' } }

If the data is not present, you can provide an default value, which will be return instead.
Otherwise an Error is thrown.

In [21]:
// Data that is not present
dataDistributor.pullData("not_contained", "the default value, if the datapoint is not present is located here!")

'the default value, if the datapoint is not present is located here!'

In [22]:
// Accessing data with a pattern
dataDistributor.patternbasedPullData("+/status/state")

[
  { path: 'robot/status/state', data: 'moving' },
  { path: 'sensor/status/state', data: 'waiting' }
]

As you can see, we receive an array, which contains the object containing:
- the `path` of the data
- the `data`

Additionally you are able to manipulate data using a path with wildcards, which will trigger our subscriptions as well:

In [24]:
dataDistributor.patternBasedPush("+/status/state", "manipulated!")

The Robot-Status is { state: 'manipulated!', speed: 1 }


If we look at the data again, we have manipulated the state of the sensor and robot

In [25]:
dataDistributor.data

{
  robot: { status: { state: 'manipulated!', speed: 1 } },
  sensor: { status: { state: 'manipulated!' } }
}

Like the `eventDistributor` the `dataDistributor` shares those changes with all runtimes in the network. For this purpose please check the docu of the [`eventDistributor`](./06-event-distributor.md)